Bu çalışmada Keşifsel Veri Analizi yöntemi ile Kalp Hastalıklarını ve Kalp Hastalıkları ile diğer hastalıklar arasındaki ilişkileri anlamaya çalışacağız. Keşifsel Veri Analizi işleminden sonra(yani verileri keşfettikten sonra) methodlarımızın sonuçlarını tahmin etmek için k-NN(k- En Yakın Komşular) algoritmasını uygulayacağız. Daha sonra diğer algoritmalarıda uygulayarak tahminler yapmaya çalışacağız.

In [ ]:
import warnings 
warnings.filterwarnings('ignore')

###  **Şimdi İhtiyacımız olan Python Kütüphanelerini çalışma alanımıza dahil edelim.**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

### ***Kullanacağımız veri setini yükleyelim.(Veri Seti Kaggle'da mevcut olduğu için basit import kodu ile veri setini yüklüyoruz.) ***

In [ ]:
vc = pd.read_csv('../input/heart.csv')
vc.head()

Veri Setinde bulunan sütunlar ve açıklamaları; <br>

**1.age(Yaşlar)** <br>
**2.sex(cinsiyet)(1 = Erkek, 0 = Kadın) <br>
**3.cp**:chest pain type(göğüs ağrısı türü)** <br>
**4.trstbps:** resting blood pressure(in mm Hg on admission to the hospital- kan basıncı) <br>
**5.chol:** serum cholestoral in mg/dl(Serum kollestrolü) <br>
**6.fbs:** (fasting blood sugar > 120 mg/dl)(1 = true; 0 = false) <br>
**7.restecg:** resting electrocardiographic results(Elektrokardiografik sonuçların dinlenmesi) <br>
**8.thalach:** maximum hear rate achieved(maksimum kalp atış hızı) <br>
**9.exang:** exercise induced angina(1 = yes; 0 = no) <br>
**10.oldpeak:** ST depression induced by exercise relative to test  (Teste göre indüklenen depresyon)<br>
**11.slope:** the slope of the peak exercise ST segment (Tepe egzersizi ST segmentinin eğimi) <br>
**12.ca:** number of major vessels(0-3) colored by flourosopy(floroskopi ile renklendirilmiş ana damarların (0-3) sayısı) <br>
**13.thal:** 3 = normal; 6 = fixed defect; 7 = reversable defect 
**14.target:** 1 or 0 




### **Veri Setinin özellikleri**

In [ ]:
vc.info()

### **Elimizdeki Verisetinin matris boyutu**

In [ ]:
print('Veri Setindeki Satır Sayısı: ', vc.shape[0])
print('Veri Setindeki Sütun Sayısı: ', vc.shape[1])

Ya da aşağıdaki gibi satır ve sütun sayısını kontrol edebiliriz.

In [ ]:
vc.shape

### **Veri Setimizde eksik(Null) değerler varmı  onu kontrol ediyoruz. Çünkü null değerler sonuçlarımızı etkileyebilir.**

In [ ]:
vc.isnull().sum()

Yukarıdaki sonuçlardan görüleceği üzere veri setimizde boş değer(Null) bulunmamaktadır bu güzel bir durum.

In [ ]:
vc.describe()

** Yukarıdaki veri setinde bulunan değerleri kısaca açıklayacak olursak; <br>
**1.Count:** Veri setimizde bulunan satır satısını bize verir.(Bu satırlar tamamen dolu olan sayıdır. <br>
**2.Mean:** İlgili sütunda bulunan değerlerin ortalamasını verir. <br>
**3.Std:** Özelliklerin  Standart Dağılım bilgisini verir. <br>
**4.Min:** Veri Setinde ilgili sütundaki özellik(feature) değerlerin en küçüğünün bilgisini verir. <br>
**5.%25, %50 ve %75 oranlarını verir. <br>
**Max:** Özelliğin en büyük değerini verir. 

### **Farklı niteliklerin özelliklerini kontrol edelim**


####**1.Cinsiyet(Sex)**

In [ ]:
Erkek =len(vc[vc['sex'] == 1])
Kadın = len(vc[vc['sex']== 0])

plt.figure(figsize=(12,8))

# Veriyi görselleştirelim
labels = 'Erkek','Kadın'
sizes = [Erkek,Kadın]
colors = ['skyblue', 'yellowgreen']
explode = (0, 0)  
 
# Plot
plt.pie(sizes, explode=explode, labels=labels, colors=colors,
autopct='%1.1f%%', shadow=True, startangle=50)
 
plt.axis('equal')
plt.show()

### **2. Göğüs Ağrısı türü**

In [ ]:
plt.figure(figsize=(8, 6))

labels = 'Göğüs Ağrısı Türü:0', 'Göğüs Ağrısı Türü:1', 'Göğüs Ağrısı Türü: :2', 'Göğüs Ağrısı Türü:3'
sizes = [len(vc[vc['cp'] == 0]),
         len(vc[vc['cp'] == 1]),
         len(vc[vc['cp'] == 2]),
         len(vc[vc['cp'] == 3])]
colors = ['skyblue', 'yellowgreen', 'orange', 'gold']
explode = (0, 0, 0, 0)

#Çizdirelim
plt.pie(sizes, explode = explode, labels = labels, colors = colors,
       autopct = '%1.1f%%', shadow = True, startangle = 180)
plt.axis('equal')
plt.show()

#### **3. fbs: (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)**

In [ ]:
plt.figure(figsize=(12,8))

etiketler = 'Kan Şekeri  < 120 mg/dl', 'Kan şekeri > 120 mg/dl'
boyutlar =[len(vc[vc['fbs'] == 0]),
          len(vc[vc['cp'] == 1])]
renkler =['skyblue', 'yellowgreen', 'orange', 'gold']
explode = (0, 1, 0)

#Şimdi Şekli Çizdirelim
plt.pie(boyutlar, explode = explode, labels = etiketler, colors = renkler, autopct = '%1.1f%%', shadow = True, startangle = 180)

plt.axis('equal')
plt.show()

#### **4.exang: exercise induced angina (1 = yes; 0 = no)**

In [ ]:
plt.figure(figsize =(12,8))

etiketler = 'Hayır', 'Evet'
boyutlar = [len(vc[vc['exang'] == 0]),
           len(vc[vc['exang'] == 1])]
renkler = ['skyblue', 'yellowgreen']
explode = (0.1, 0)

plt.pie(boyutlar, explode = explode, labels = etiketler, colors = renkler, autopct = '%1.1f%%', shadow = True, startangle=90)

plt.axis('equal')
plt.show()

### Keşifsel Veri Analizi (Exploratory Data Analysis)

Veri keşfine çıkalım.

In [ ]:
sns.set_style('whitegrid')

#### **1.Sıcaklık Haritası **(Heatmap)

In [ ]:
plt.figure(figsize=(16,8))
sns.heatmap(vc.corr(), annot = True, cmap = 'coolwarm', linewidths = .1)
plt.show()

#### Değişkenlerin dağılımını gösterir çubuk grafiği çizdirelim.

#### **1.Erişilen Maksimum kalp hızı oranları ** (thalach: maximum heart rate achieved)

In [ ]:
sns.distplot(vc['thalach'], kde = False, bins = 30, color = 'violet')

#### **2.chol: serum cholestoral in mg/dl **

In [ ]:
sns.distplot(vc['chol'],  kde = False, bins = 30, color = 'red')
plt.show()

#### ** Dinlenme halindeki kan basıncı ** (3. trestbps: resting blood pressure (in mm Hg on admission to the hospital)

In [ ]:
sns.distplot(vc['trestbps'], kde = False, bins = 30, color = 'blue')
plt.show()

#### 4.Yaşlara göre Kalp hastalığı oranları

In [ ]:
plt.figure(figsize = (16,8))
sns.countplot(x = 'age', data = vc, hue = 'target', palette = 'GnBu')
plt.show()

Not: Yukarıdaki grafikten anlaşılacağı üzere kalp krizi riski 55 Yaşına kadar Erkeklerde(1), 55 Yaşından sonra ise Kadın(0)'larda daha çok görülmektedir.

#### **5.Scatterplot for thalach vs. chol **

In [ ]:
plt.figure(figsize =(12,8))
sns.scatterplot(x = 'chol', y = 'thalach', data = vc, hue = 'target')
plt.show()

#### **6.Scatterplot for thalach vs. trestbps **

In [ ]:
plt.figure(figsize=(12, 8))
sns.scatterplot(x = 'trestbps', y = 'thalach', data = vc, hue = 'target')
plt.show()

### TAHMİNLER YAPALIM

**Veri setimizi ikiye (Eğitim ve Test) setlerine ayırıyoruz.

In [ ]:
X = vc.drop('target', axis = 1)
y = vc['target']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state = 42)

**Özellikleri ön işleme tabi tutarak ölçeklediriyoruz.**

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler(feature_range =(0,1))

X_train_scaled = scaler.fit_transform(X_train)
X_train = pd.DataFrame(X_train_scaled)

X_test_scaled = scaler.fit_transform(X_test)
X_test = pd.DataFrame(X_test_scaled)

En iyi parametreleri seçmek için GridSearchCv ve k-NN Algoritmasını uyguluyoruz**Implementing GridSearchCv to select best parameters and applying k-NN Algorithm**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
knn = KNeighborsClassifier()
params = {'n_neighbors':[i for i in range(1, 33, 2)]}

In [ ]:
model = GridSearchCV(knn, params, cv = 10)

In [ ]:
model.fit(X_train, y_train)
model.best_params_

Şimdi Tahminler Yapalım

In [ ]:
predict = model.predict(X_test)
predict

**Tahmin yapalım bakalım ***

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
print('Doğruluk oranı:', accuracy_score(y_test, predict))
print('k-NN algoritmasını kullanarak yapılan modelin tahmin oranı :', 
     round(accuracy_score(y_test, predict), 5)*100, '%')

**Modelimizin Doğruluğunu kontrol edelim**

**Confusion Matrix**

In [ ]:
cnf_matrix = confusion_matrix(y_test, predict)
cnf_matrix

In [ ]:
class_names = [0,1]
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)

# Yeni bir sıcaklık haritası oluşturalım
sns.heatmap(pd.DataFrame(cnf_matrix), annot = True, cmap = 'YlGnBu',
           fmt ='g')
ax.xaxis.set_label_position('top')
plt.tight_layout()
plt.title('Karşılaştırma matrisi', y = 1.1)
plt.ylabel('Gerçek değerler')
plt.xlabel('Tahmin edilen değerler')
plt.show()

**Suggestions are welcome**